# 📓 4. Run Queries
## Query Performance Benchmarking for TC.11.x–TC.13.x

### 🔗 Ensure `BenchmarkLakehouse` is connected as a data source before running.

Assumes synthetic data, initial load, and updates have been completed.
All queries are run against the target table in BenchmarkLakehouse and BenchmarkWarehouse.
Join queries are excluded (single table per location).


In [ ]:
import time
from pyspark.sql.functions import col

In [ ]:
# Paths and table names
target_lakehouse = "BenchmarkLakehouse"
target_warehouse = "BenchmarkWarehouse"
row_count = 10000  # Update as appropriate

delta_tables = {
    "refresh": "delta_refresh_load",
    "compare": "delta_compare_load",
    "increment": "delta_increment_load"
}
warehouse_tables = {
    "refresh": "wh_table_refresh_load",
    "compare": "wh_table_compare_load",
    "increment": "wh_table_increment_load"
}

# Choose table to query (usually 'increment' for event log)
lakehouse_table = f"{target_lakehouse}.{delta_tables['increment']}"
warehouse_table = f"{target_warehouse}.dbo.{warehouse_tables['increment']}"

In [ ]:
# Read Delta table from Lakehouse
df_lakehouse = spark.read.table(lakehouse_table)

## Query Types
- **Filter Query**: Select rows with a specific category value.
- **Aggregate Query**: Group by category, aggregate numeric columns.
- **Batch Query**: Select update events in a specific time window.
- **Top-N Query**: Retrieve top N rows by a numeric column.

*(Join queries are excluded—only one table in each target location.)*

In [ ]:
# Define query functions
def run_filter_query(df):
    # Example: Filter by category
    return df.filter(col("cat_1") == "A").count()

def run_aggregate_query(df):
    # Example: Group by category, aggregate numerics
    return df.groupBy("cat_1").agg({"num_1": "avg", "num_2": "max"}).count()

def run_batch_query(df):
    # Example: Select events by update_type and time window
    return df.filter((col("update_type") == "update") & (col("ts_1") > "2025-01-01")).count()

def run_topn_query(df, n=10):
    # Example: Top-N by numeric value
    return df.orderBy(col("num_1").desc()).limit(n).count()

In [ ]:
# Performance logging utility
def log_query_perf(query_func, df, description):
    start = time.time()
    result = query_func(df)
    elapsed = time.time() - start
    print(f"{description}: {elapsed:.3f}s (Rows: {result})")
    return {"query": description, "rows": result, "time_s": elapsed}

In [ ]:
# Run queries on Lakehouse Delta table
lakehouse_metrics = []
lakehouse_metrics.append(log_query_perf(run_filter_query, df_lakehouse, "Lakehouse Filter cat_1 == 'A'"))
lakehouse_metrics.append(log_query_perf(run_aggregate_query, df_lakehouse, "Lakehouse Aggregate by cat_1"))
lakehouse_metrics.append(log_query_perf(run_batch_query, df_lakehouse, "Lakehouse Batch update_type == update, ts_1 > '2025-01-01'"))
lakehouse_metrics.append(log_query_perf(run_topn_query, df_lakehouse, "Lakehouse Top 10 num_1"))

In [ ]:
# Read Warehouse table (for Spark SQL endpoint, not as DataFrame)
from com.microsoft.spark.fabric import Constants
df_warehouse = spark.read.synapsesql(warehouse_table)

In [ ]:
# Run queries on Warehouse table
warehouse_metrics = []
warehouse_metrics.append(log_query_perf(run_filter_query, df_warehouse, "Warehouse Filter cat_1 == 'A'"))
warehouse_metrics.append(log_query_perf(run_aggregate_query, df_warehouse, "Warehouse Aggregate by cat_1"))
warehouse_metrics.append(log_query_perf(run_batch_query, df_warehouse, "Warehouse Batch update_type == update, ts_1 > '2025-01-01'"))
warehouse_metrics.append(log_query_perf(run_topn_query, df_warehouse, "Warehouse Top 10 num_1"))

In [ ]:
# Display metrics as table
import pandas as pd

all_metrics = lakehouse_metrics + warehouse_metrics
metrics_df = pd.DataFrame(all_metrics)
display(metrics_df)

# Print completion message
print("Query performance benchmarking complete. Metrics above can be visualized in the next step.")